In [2]:
import escher
from escher import Builder
import cobra
from time import sleep

## Load the organism file.

In [3]:
model = cobra.io.load_json_model("H:/ROBOT_SCIENTIST/E_coli/iML1515.json")
model

Name,iML1515
Memory address,22573ff4c20
Number of metabolites,1877
Number of reactions,2712
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Map conversion functions
> These are helper functions.

In [1]:
"""
Create a custom iML1515 Nucleotide Metabolism map by adapting the iJO1366 map.

This script:
1. Downloads the iJO1366.Nucleotide metabolism map
2. Builds a mapping between iJO1366 and iML1515 reactions
3. Replaces all iJO1366 reaction IDs in the map with iML1515 IDs
4. Saves the new map for future use
"""

import cobra
import json
import os
import requests
from escher import Builder

# Helper functions
def load_model(path):
    """Load a COBRA model from SBML or JSON."""
    if path.endswith(".xml") or path.endswith(".sbml"):
        return cobra.io.read_sbml_model(path)
    elif path.endswith(".json"):
        return cobra.io.load_json_model(path)
    else:
        raise ValueError("Unsupported model format")

def normalize_stoichiometry(reaction):
    """Return a frozenset of metabolite:coefficient pairs for comparison."""
    return frozenset((met.id, coeff) for met, coeff in reaction.metabolites.items())

def build_reaction_mapping_ijo_to_iml(model_ijo1366, model_iml1515):
    """
    Map reactions from iJO1366 to iML1515 based on:
    1. Exact ID match
    2. Stoichiometry match
    Returns: dict {ijo1366_id: iml1515_id}
    """
    mapping = {}
    iml_stoich_map = {normalize_stoichiometry(r): r.id for r in model_iml1515.reactions}
    
    for r_ijo in model_ijo1366.reactions:
        # Case 1: Exact ID match
        if r_ijo.id in model_iml1515.reactions:
            mapping[r_ijo.id] = r_ijo.id
            continue
        
        # Case 2: Stoichiometry match
        stoich = normalize_stoichiometry(r_ijo)
        if stoich in iml_stoich_map:
            mapping[r_ijo.id] = iml_stoich_map[stoich]
    
    return mapping

def create_iml1515_map(source_map_name, iml1515_model_path, ijo1366_model_path, output_path):
    """
    Create a new Escher map for iML1515 by adapting an iJO1366 map.
    
    Args:
        source_map_name: Name of the iJO1366 map (e.g., 'iJO1366.Nucleotide metabolism')
        iml1515_model_path: Path to iML1515 model
        ijo1366_model_path: Path to iJO1366 model
        output_path: Where to save the new iML1515 map
    """
    print(f"Loading models...")
    model_iml = load_model(iml1515_model_path)
    model_ijo = load_model(ijo1366_model_path)
    print(f"  iML1515: {len(model_iml.reactions)} reactions")
    print(f"  iJO1366: {len(model_ijo.reactions)} reactions")
    
    print(f"\nDownloading {source_map_name} map...")
    # Download the map directly from Escher's GitHub
    map_url = f"https://escher.github.io/1-0-0/6/maps/Escherichia%20coli/{source_map_name.replace(' ', '%20')}.json"
    print(f"  URL: {map_url}")
    
    response = requests.get(map_url)
    if response.status_code != 200:
        raise ValueError(f"Failed to download map. Status code: {response.status_code}")
    
    map_data = response.json()
    print(f"  Map downloaded successfully")
    print(f"  Map type: {type(map_data)}")
    
    print(f"\nBuilding reaction mapping from iJO1366 to iML1515...")
    reaction_mapping = build_reaction_mapping_ijo_to_iml(model_ijo, model_iml)
    print(f"  Found {len(reaction_mapping)} mapped reactions")
    
    print(f"\nTranslating map reaction IDs...")
    
    translated_count = 0
    not_found_count = 0
    
    # Find the reactions dictionary in the map structure
    map_object = None
    if isinstance(map_data, list):
        print(f"  Map is a list with {len(map_data)} items")
        # Try to find the map object in the list
        for i, item in enumerate(map_data):
            if isinstance(item, dict):
                print(f"    Item {i}: dict with keys {list(item.keys())[:5]}")
                if 'reactions' in item:
                    map_object = item
                    print(f"    Found reactions in item {i}")
                    break
    elif isinstance(map_data, dict):
        print(f"  Map is a dict with keys: {list(map_data.keys())[:10]}")
        map_object = map_data
    
    if map_object and 'reactions' in map_object:
        reactions_dict = map_object['reactions']
        print(f"  Found {len(reactions_dict)} reactions in map")
        
        for rxn_key, rxn_data in reactions_dict.items():
            if isinstance(rxn_data, dict) and 'bigg_id' in rxn_data:
                old_id = rxn_data['bigg_id']
                if old_id in reaction_mapping:
                    new_id = reaction_mapping[old_id]
                    rxn_data['bigg_id'] = new_id
                    translated_count += 1
                else:
                    not_found_count += 1
        
        print(f"  Translated: {translated_count} reactions")
        print(f"  Not found in iML1515: {not_found_count} reactions")
    else:
        print("  ERROR: Could not find reactions in map structure!")
        raise ValueError("Map structure doesn't contain reactions")
    
    print(f"\nSaving new map to {output_path}...")
    with open(output_path, 'w') as f:
        json.dump(map_data, f, indent=2)
    
    print(f"\n✓ Success! Created iML1515 map at: {output_path}")
    return output_path


# Load the experiment data

> If the data files are created by the pipeline, you must have 'df_flux.csv' and 'growth_rates.csv', created in different folders based on starin, averaging, etc.
>
> Check the parameters below to change the data source selection.

In [16]:
import pandas as pd

strain = "WT"
replication = "no_replicates"
gr_column= "mv_mu_max"
# replication = "replicates"
# gr_column= "mv_mu_max"
# replication = "post_replicates"
# gr_column= "mv_mu_max_mean"
experiment = "mediabotJLF2"
# replication = "post_replicates"
# gr_column= "mv_mu_max_mean"
# replication = "replicates"
# gr_column= "mv_mu_max"
exp_data_path = f"H:/ROBOT_SCIENTIST/E_coli/Growth_rates/2025-10-31-27/processed/{replication}/{experiment}/AMN_dataset/"
# exp_data_path = f"H:/ROBOT_SCIENTIST/E_coli/Growth_rates/2025-10-31-27/processed/{replication}_STRAINS/{strain}/AMN_dataset/"
exp_data = pd.read_csv(exp_data_path + "df_flux.csv")
#
growth_data_path = f"H:/ROBOT_SCIENTIST/E_coli/Growth_rates/2025-10-31-27/processed/{replication}/{experiment}/"
growth_data = pd.read_csv(growth_data_path + "growth_rates.csv")
#
gr_column = "mv_mu_max"
# Filter growth_data to match the rows in exp_data 
# Since exp_data was created from growth_data where success=='ok', we need to apply the same filter
# OR simply take the first len(exp_data) rows if they're already aligned
growth_data = growth_data.reset_index(drop=True)
growth_data = growth_data.loc[growth_data['success'], :]
exp_data = exp_data.reset_index(drop=True)        
# Remove growth rate column from growth_data to avoid duplicates
growth_data = growth_data.drop(columns=[gr_column])                
# Now join them - they should have the same number of rows
combinded_data = pd.concat([exp_data.reset_index(drop=True), growth_data.reset_index(drop=True)],
                            axis=1)
# combinded_data

# Solve the FBA for every records

In [5]:

preds = []
status = []
fluxes = []
shadow_prices = []
i = 0
for _, row in exp_data.iterrows():
    with model:
        # Set the medium according to the experimental data
        medium = model.medium.copy()
        for rxn_id in medium:
            medium[rxn_id] = 0.0  # Set high uptake rates for all exchange reactions
        for rxn_id in exp_data.columns:
            if rxn_id != gr_column:
                medium[rxn_id] = row.get(rxn_id, 0.0)  # Set uptake rates based on experimental data
        model.medium = medium
        # Perform FBA
        # model.objective = 'BIOMASS_Ec_iML1515_WT_75p37M'
        model.objective = 'BIOMASS_Ec_iML1515_core_75p37M'                
        solution = model.optimize()        
        objective_value = solution.objective_value
        solution_status = solution.status
        solution_fluxes = solution.fluxes
        solution_shadow_prices = solution.shadow_prices
        preds.append(objective_value)
        status.append(solution_status)
        fluxes.append(solution_fluxes)
        shadow_prices.append(solution_shadow_prices)

c:\Users\rh2310\projects\FBA_tools\.venv\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


# IMPORTANT
## In the following, the 'solution_index' is a variable that selects the record in the 'combinded_data' dataframe will be plotted. You can check the 'combinded_data' dataframe, then, select the relevant index to study furthur.

# Core metabolism

In [6]:
solution_index = 1  # Change this index to visualize different solutions
assert status[solution_index] == 'optimal', f"Solution at index {solution_index} is not optimal."
assert solution_index < len(fluxes), f"Solution index {solution_index} is out of bounds for fluxes list of length {len(fluxes)}."

builder = Builder(    
    map_name='e_coli_core.Core metabolism',  
    model=model,
    reaction_data=fluxes[solution_index],
    metabolite_data=shadow_prices[solution_index]
)

# Simplify the map by hiding some labels
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False

# Set color scale
builder.reaction_scale_preset = 'GaBuRd'

# Make all the arrows three times as thick
# builder.reaction_scale = [
#     {k: v * 3 if k == 'size' else v for k, v in x.items()}
#     for x in builder.reaction_scale
# ]

builder

Builder(hide_all_labels=False, hide_secondary_metabolites=True, metabolite_data={'octapb_c': np.float64(0.0), …

# Nucleotide Metabolism

In [7]:
# === Configuration ===
iml1515_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.json"
ijo1366_path = "H:/ROBOT_SCIENTIST/E_coli/iJO1366.xml"
output_map_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_metabolism.json"

# === Create the map (only run once) ===
if not os.path.exists(output_map_path):
    create_iml1515_map(
        source_map_name='iJO1366.Nucleotide metabolism',
        iml1515_model_path=iml1515_path,
        ijo1366_model_path=ijo1366_path,
        output_path=output_map_path
    )
else:
    print(f"Map already exists at: {output_map_path}")
    print("Delete the file if you want to regenerate it.")


Map already exists at: H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_metabolism.json
Delete the file if you want to regenerate it.


In [8]:
# Use the custom iML1515 Nucleotide Metabolism map
solution_index = 1  # Change this to visualize different solutions

# Verify solution is optimal
assert status[solution_index] == 'optimal', f"Solution at index {solution_index} is not optimal."
assert solution_index < len(fluxes), f"Index {solution_index} out of bounds for fluxes list."

# Create the builder with your custom iML1515 map
builder = Builder(
    map_json='H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_metabolism.json',  # Your custom map!
    model=model,  # Your iML1515 model - no translation needed!
    reaction_data=fluxes[solution_index],  # Your iML1515 flux data - works directly!
    metabolite_data=shadow_prices[solution_index]
)

# Customize appearance
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False
builder.reaction_scale_preset = 'GaBuRd'

# Display
builder

Builder(hide_all_labels=False, hide_secondary_metabolites=True, metabolite_data={'octapb_c': np.float64(0.0), …

# Fatty acid biosynthesis

In [9]:
# === Configuration ===
iml1515_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.json"
ijo1366_path = "H:/ROBOT_SCIENTIST/E_coli/iJO1366.xml"
output_map_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.Fatty_acid_biosynthesis_saturated.json"

# === Create the map (only run once) ===
if not os.path.exists(output_map_path):
    create_iml1515_map(
        source_map_name='iJO1366.Fatty acid biosynthesis (saturated)',
        iml1515_model_path=iml1515_path,
        ijo1366_model_path=ijo1366_path,
        output_path=output_map_path
    )
else:
    print(f"Map already exists at: {output_map_path}")
    print("Delete the file if you want to regenerate it.")

#
solution_index = 1  # Change this to visualize different solutions

# Verify solution is optimal
assert status[solution_index] == 'optimal', f"Solution at index {solution_index} is not optimal."
assert solution_index < len(fluxes), f"Index {solution_index} out of bounds for fluxes list."

# Create the builder with your custom iML1515 map
builder = Builder(
    map_json='H:/ROBOT_SCIENTIST/E_coli/iML1515.Fatty_acid_biosynthesis_saturated.json',  # Your custom map!
    model=model,  # Your iML1515 model - no translation needed!
    reaction_data=fluxes[solution_index],  # Your iML1515 flux data - works directly!
    metabolite_data=shadow_prices[solution_index]
)

# Customize appearance
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False
builder.reaction_scale_preset = 'GaBuRd'

# Display
builder

Map already exists at: H:/ROBOT_SCIENTIST/E_coli/iML1515.Fatty_acid_biosynthesis_saturated.json
Delete the file if you want to regenerate it.


Builder(hide_all_labels=False, hide_secondary_metabolites=True, metabolite_data={'octapb_c': np.float64(0.0), …

# Nucleotide and histidine biosynthesis

In [10]:
# === Configuration ===
iml1515_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.json"
ijo1366_path = "H:/ROBOT_SCIENTIST/E_coli/iJO1366.xml"
output_map_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_and_histidine_biosynthesis.json"

# === Create the map (only run once) ===
if not os.path.exists(output_map_path):
    create_iml1515_map(
        source_map_name='iJO1366.Nucleotide and histidine biosynthesis',
        iml1515_model_path=iml1515_path,
        ijo1366_model_path=ijo1366_path,
        output_path=output_map_path
    )
else:
    print(f"Map already exists at: {output_map_path}")
    print("Delete the file if you want to regenerate it.")

#
solution_index = 1  # Change this to visualize different solutions

# Verify solution is optimal
assert status[solution_index] == 'optimal', f"Solution at index {solution_index} is not optimal."
assert solution_index < len(fluxes), f"Index {solution_index} out of bounds for fluxes list."

# Create the builder with your custom iML1515 map
builder = Builder(
    map_json='H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_and_histidine_biosynthesis.json',  # Your custom map!
    model=model,  # Your iML1515 model - no translation needed!
    reaction_data=fluxes[solution_index],  # Your iML1515 flux data - works directly!
    metabolite_data=shadow_prices[solution_index]
)

# Customize appearance
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False
builder.reaction_scale_preset = 'GaBuRd'

# Display
builder

Map already exists at: H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_and_histidine_biosynthesis.json
Delete the file if you want to regenerate it.


Builder(hide_all_labels=False, hide_secondary_metabolites=True, metabolite_data={'octapb_c': np.float64(0.0), …

# Central metabolism

In [11]:
# === Configuration ===
iml1515_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.json"
ijo1366_path = "H:/ROBOT_SCIENTIST/E_coli/iJO1366.xml"
output_map_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.Central_metabolism.json"

# === Create the map (only run once) ===
if not os.path.exists(output_map_path):
    create_iml1515_map(
        source_map_name='iJO1366.Central metabolism',
        iml1515_model_path=iml1515_path,
        ijo1366_model_path=ijo1366_path,
        output_path=output_map_path
    )
else:
    print(f"Map already exists at: {output_map_path}")
    print("Delete the file if you want to regenerate it.")

#
solution_index = 1  # Change this to visualize different solutions

# Verify solution is optimal
assert status[solution_index] == 'optimal', f"Solution at index {solution_index} is not optimal."
assert solution_index < len(fluxes), f"Index {solution_index} out of bounds for fluxes list."

# Create the builder with your custom iML1515 map
builder = Builder(
    map_json='H:/ROBOT_SCIENTIST/E_coli/iML1515.Central_metabolism.json',  # Your custom map!
    model=model,  # Your iML1515 model - no translation needed!
    reaction_data=fluxes[solution_index],  # Your iML1515 flux data - works directly!
    metabolite_data=shadow_prices[solution_index]
)

# Customize appearance
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False
builder.reaction_scale_preset = 'GaBuRd'

# Display
builder

Map already exists at: H:/ROBOT_SCIENTIST/E_coli/iML1515.Central_metabolism.json
Delete the file if you want to regenerate it.


Builder(hide_all_labels=False, hide_secondary_metabolites=True, metabolite_data={'octapb_c': np.float64(0.0), …

# Fatty acid beta-oxidation

In [12]:
# === Configuration ===
iml1515_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.json"
ijo1366_path = "H:/ROBOT_SCIENTIST/E_coli/iJO1366.xml"
output_map_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.Fatty_acid_beta_oxidation.json"

# === Create the map (only run once) ===
if not os.path.exists(output_map_path):
    create_iml1515_map(
        source_map_name='iJO1366.Fatty acid beta-oxidation',
        iml1515_model_path=iml1515_path,
        ijo1366_model_path=ijo1366_path,
        output_path=output_map_path
    )
else:
    print(f"Map already exists at: {output_map_path}")
    print("Delete the file if you want to regenerate it.")

#
solution_index = 1  # Change this to visualize different solutions

# Verify solution is optimal
assert status[solution_index] == 'optimal', f"Solution at index {solution_index} is not optimal."
assert solution_index < len(fluxes), f"Index {solution_index} out of bounds for fluxes list."

# Create the builder with your custom iML1515 map
builder = Builder(
    map_json='H:/ROBOT_SCIENTIST/E_coli/iML1515.Fatty_acid_beta_oxidation.json',  # Your custom map!
    model=model,  # Your iML1515 model - no translation needed!
    reaction_data=fluxes[solution_index],  # Your iML1515 flux data - works directly!
    metabolite_data=shadow_prices[solution_index]
)

# Customize appearance
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False
builder.reaction_scale_preset = 'GaBuRd'

# Display
builder

Map already exists at: H:/ROBOT_SCIENTIST/E_coli/iML1515.Fatty_acid_beta_oxidation.json
Delete the file if you want to regenerate it.


Builder(hide_all_labels=False, hide_secondary_metabolites=True, metabolite_data={'octapb_c': np.float64(0.0), …

In [13]:
# To see all available maps:
import escher
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [14]:
builder = Builder(
    map_name='iJO1366.Nucleotide metabolism',   
    model=model,
    reaction_data=fluxes[solution_index],
    metabolite_data=shadow_prices[solution_index]
)

In [15]:
"""
Create a custom iML1515 Nucleotide Metabolism map by adapting the iJO1366 map.

This script:
1. Downloads the iJO1366.Nucleotide metabolism map
2. Builds a mapping between iJO1366 and iML1515 reactions
3. Replaces all iJO1366 reaction IDs in the map with iML1515 IDs
4. Saves the new map for future use
"""

import cobra
import json
import os
from escher import Builder

# Helper functions
def load_model(path):
    """Load a COBRA model from SBML or JSON."""
    if path.endswith(".xml") or path.endswith(".sbml"):
        return cobra.io.read_sbml_model(path)
    elif path.endswith(".json"):
        return cobra.io.load_json_model(path)
    else:
        raise ValueError("Unsupported model format")

def normalize_stoichiometry(reaction):
    """Return a frozenset of metabolite:coefficient pairs for comparison."""
    return frozenset((met.id, coeff) for met, coeff in reaction.metabolites.items())

def build_reaction_mapping_ijo_to_iml(model_ijo1366, model_iml1515):
    """
    Map reactions from iJO1366 to iML1515 based on:
    1. Exact ID match
    2. Stoichiometry match
    Returns: dict {ijo1366_id: iml1515_id}
    """
    mapping = {}
    iml_stoich_map = {normalize_stoichiometry(r): r.id for r in model_iml1515.reactions}
    
    for r_ijo in model_ijo1366.reactions:
        # Case 1: Exact ID match
        if r_ijo.id in model_iml1515.reactions:
            mapping[r_ijo.id] = r_ijo.id
            continue
        
        # Case 2: Stoichiometry match
        stoich = normalize_stoichiometry(r_ijo)
        if stoich in iml_stoich_map:
            mapping[r_ijo.id] = iml_stoich_map[stoich]
    
    return mapping

def create_iml1515_map(source_map_name, iml1515_model_path, ijo1366_model_path, output_path):
    """
    Create a new Escher map for iML1515 by adapting an iJO1366 map.
    
    Args:
        source_map_name: Name of the iJO1366 map (e.g., 'iJO1366.Nucleotide metabolism')
        iml1515_model_path: Path to iML1515 model
        ijo1366_model_path: Path to iJO1366 model
        output_path: Where to save the new iML1515 map
    """
    print(f"Step 1: Loading models...")
    model_iml = load_model(iml1515_model_path)
    model_ijo = load_model(ijo1366_model_path)
    print(f"  iML1515: {len(model_iml.reactions)} reactions")
    print(f"  iJO1366: {len(model_ijo.reactions)} reactions")
    
    print(f"\nStep 2: Downloading {source_map_name} map...")
    temp_builder = Builder(map_name=source_map_name)
    map_data = temp_builder.map_json
    
    print(f"\nStep 3: Building reaction mapping from iJO1366 to iML1515...")
    reaction_mapping = build_reaction_mapping_ijo_to_iml(model_ijo, model_iml)
    print(f"  Found {len(reaction_mapping)} mapped reactions")
    
    print(f"\nStep 4: Translating map reaction IDs...")
    # The map has a structure: map_data[1]['reactions'] contains reaction objects
    # Each reaction has properties like 'bigg_id', 'segments', etc.
    
    translated_count = 0
    not_found_count = 0
    
    if isinstance(map_data, list) and len(map_data) > 0:
        map_object = map_data[0] if isinstance(map_data[0], dict) else map_data[1]
    else:
        map_object = map_data
    
    if 'reactions' in map_object:
        for rxn_key, rxn_data in map_object['reactions'].items():
            if 'bigg_id' in rxn_data:
                old_id = rxn_data['bigg_id']
                if old_id in reaction_mapping:
                    new_id = reaction_mapping[old_id]
                    rxn_data['bigg_id'] = new_id
                    translated_count += 1
                else:
                    # Keep original but mark it
                    not_found_count += 1
    
    print(f"  Translated: {translated_count} reactions")
    print(f"  Not found in iML1515: {not_found_count} reactions")
    
    print(f"\nStep 5: Saving new map to {output_path}...")
    with open(output_path, 'w') as f:
        json.dump(map_data, f, indent=2)
    
    print(f"\n✓ Success! Created iML1515 map at: {output_path}")
    return output_path


# === Configuration ===
iml1515_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.json"
ijo1366_path = "H:/ROBOT_SCIENTIST/E_coli/iJO1366.xml"
output_map_path = "H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_metabolism.json"

# === Create the map (only run once) ===
if not os.path.exists(output_map_path):
    create_iml1515_map(
        source_map_name='iJO1366.Nucleotide metabolism',
        iml1515_model_path=iml1515_path,
        ijo1366_model_path=ijo1366_path,
        output_path=output_map_path
    )
else:
    print(f"Map already exists at: {output_map_path}")
    print("Delete the file if you want to regenerate it.")

# === Now use your new map! ===
print("\n" + "="*60)
print("USAGE: Now you can use your custom map like this:")
print("="*60)
print("""
solution_index = 1
builder = Builder(
    map_json='H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_metabolism.json',
    model=model,  # Your iML1515 model
    reaction_data=fluxes[solution_index],
    metabolite_data=shadow_prices[solution_index]
)
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False
builder.reaction_scale_preset = 'GaBuRd'
builder
""")

Map already exists at: H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_metabolism.json
Delete the file if you want to regenerate it.

USAGE: Now you can use your custom map like this:

solution_index = 1
builder = Builder(
    map_json='H:/ROBOT_SCIENTIST/E_coli/iML1515.Nucleotide_metabolism.json',
    model=model,  # Your iML1515 model
    reaction_data=fluxes[solution_index],
    metabolite_data=shadow_prices[solution_index]
)
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False
builder.reaction_scale_preset = 'GaBuRd'
builder

